# Document Summarization

This notebook demonstrates an application of long document summarization techniques to a work of literature.

## Setting up the environment

### Install Dependencies

Granite Kitchen comes with a bundle of dependencies that are required for notebooks. See the list of packages in its [`setup.py`](https://github.com/ibm-granite-community/granite-kitchen/blob/main/setup.py). 

In [ ]:
! pip install git+https://github.com/ibm-granite-community/granite-kitchen \
    transformers \
    torch \
    tiktoken

### Ollama

This notebook requires IBM Granite models to be served by a AI model runtime so that the models can be inferred or called. This notebook uses [Ollama](https://github.com/ollama/ollama) to serve the models.

The Ollama server can be run locally on your computer or in the notebook itself (notebook needs to be run in [Google Colab](https://colab.research.google.com)). Follow the steps in the section which best suits your needs.

#### Running Ollama Locally

If you have not already done this in the pre-work, you will need to start a local Ollama server.

Using the command line on your computer:

1. [Download and install Ollama](https://github.com/ollama/ollama?tab=readme-ov-file#ollama), if you haven't already.

    On macOS, you can use Homebrew to install with

    ```shell
    brew install ollama
    ```

1. Start the Ollama server.

    ```shell
    ollama serve
    ```

1. Pull down the Granite models you will want to use in the workshop. Larger models take more memory to run.

    ```shell
    ollama pull granite-code:3b-instruct
    ollama pull granite-code:8b-instruct
    ```

#### Running Ollama in Colab

This section is if you are not going to run the Ollama server locally on your computer. Running the Ollama server in Colab will limit the size of Granite models you can use and be _significantly_ slower when calling the Granite models.

1. Download and install Ollama in Colab

In [ ]:
!curl https://ollama.ai/install.sh | sh

2. Start the Ollama server as a background process in Colab using `nohup` and `&`

In [ ]:
import os
os.system("nohup ollama serve &")

3. Pull down the Granite models in Colab that you will use in the workshop. Larger models take more memory to run. The `granite-code:20b` model is too large for the Colab runtime environment.

In [ ]:
!ollama pull granite-code:3b-instruct
!ollama pull granite-code:8b-instruct

## Select your model

Select a Granite Code model from the [`granite-code`](https://ollama.com/library/granite-code) org on Ollama. Here we use the Ollama Langchain client to connect to the model.

In [ ]:
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="granite-code:3b-instruct")

## Download a book

Here we fetch H.D. Thoreau's "Walden" from [Project Gutenberg](https://www.gutenberg.org/) for summarization.

We have to trim it down so that it will fit in the 128k-token context window of the model.

In [ ]:
import requests
from time import sleep

# The following URL contains a text version of H.D. Thoreau's "Walden"
url = "https://www.gutenberg.org/cache/epub/205/pg205.txt"

# Get the contents
response = requests.get(url)
response.raise_for_status()
full_contents = response.text

# Extract the text of the book, leaving out the gutenberg boilerplate.
start_str = "*** START OF THE PROJECT GUTENBERG EBOOK WALDEN, AND ON THE DUTY OF CIVIL DISOBEDIENCE ***"
start_index = full_contents.index(start_str) + len(start_str)
end_str = "*** END OF THE PROJECT GUTENBERG EBOOK WALDEN, AND ON THE DUTY OF CIVIL DISOBEDIENCE ***"
end_index = full_contents.index(end_str)
book_contents = full_contents[start_index:end_index]
print("Length of book text: {} chars".format(len(book_contents)))

# We limit the text to 200k characters, which is about 57k tokens. (400k chars is ~114k tokens; 300k chars is ~86k tokens; 350k chars is ~100k tokens).
char_limit = 10000
contents = book_contents[:char_limit]
print("Length of text for summarization: {} chars".format(len(contents)))

## Count the tokens

Before sending our code to the AI model, it's crucial to understand how much of the model's capacity we're using. Language models typically have a limit on the number of tokens they can process in a single request.

Key points:
- We're using the [`granite-3B-Code-instruct-128k`](https://huggingface.co/ibm-granite/granite-3b-code-instruct-128k) model, which has a context window of 128,000 tokens.
- Tokenization can vary between models, so we use the specific tokenizer for our chosen model.

Understanding token count helps us optimize our prompts and ensure we're using the model efficiently.

In [ ]:
from transformers import AutoTokenizer

model_path = "ibm-granite/granite-3B-Code-instruct-128k"
tokenizer = AutoTokenizer.from_pretrained(model_path)
print("Your model uses the tokenizer " + type(tokenizer).__name__)

print(f"Your document has {len(tokenizer(contents, return_tensors='pt')['input_ids'][0])} tokens. ")

## Summarize the text

We construct our final prompt and send it to the AI model served by Ollama for processing.

In [ ]:
prompt = f"""
Summarize the following text from "Walden" by Henry David Thoreau:
{contents}
"""

output = model.invoke(
    prompt,
    model_kwargs={
        "max_tokens": 10000, # Set the maximum number of tokens to generate as output.
        "min_tokens": 200, # Set the minimum number of tokens to generate as output.
        "temperature": 0.75,
        "system_prompt": "You are a helpful assistant.",
        "presence_penalty": 0,
        "frequency_penalty": 0
    }
    )

print(output)

## Summary of Summaries

Here we use an iterative summarization technique to adapt to the context length of the model.

### Chunk the text

Divide the full text into smaller passages for separate processing.

In [ ]:
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document

excerpt_length = 20000
doc =  Document(page_content=book_contents[:excerpt_length], metadata={"source": "local"})
print(f"The text is {len(doc.page_content)} chars")

# Split the documents into chunks
chunk_char_limit = 1000
text_splitter = TokenTextSplitter.from_huggingface_tokenizer(tokenizer=tokenizer, chunk_size=chunk_char_limit, chunk_overlap=50)
chunks = text_splitter.split_documents([doc])
print(f"Chunk count: {len(chunks)}")

### Summarize the chunks

Here we create a separate summary of each passage. This can take a few minutes.

In [ ]:
summaries = []

for i, chunk in enumerate(chunks):
    prompt = f"""
        Summarize the following text from "Walden" by Henry David Thoreau:
        {chunk}
        """
    output = model.invoke(
        prompt,
        model_kwargs={
            "max_tokens": 10000, # Set the maximum number of tokens to generate as output.
            "min_tokens": 200, # Set the minimum number of tokens to generate as output.
            "temperature": 0.75,
            "system_prompt": "You are a helpful assistant.",
            "presence_penalty": 0,
            "frequency_penalty": 0
        }
    )
    summary = f"Summary {i+1}:\n{output}\n\n"
    summaries.append(summary)
    print(summary)

print("Summary count: " + str(len(summaries)))


### Summarize the Summaries

We signal to the model that it is receiving separate summaries of passages from an original text, and to create a unified summary of that text.

In [ ]:
summary_contents = "\n\n".join(summaries)
print(len(summary_contents))

prompt = f"""
The text of "Walden", by Henry David Thoreau, was summarized in separate passages; those passage summaries are provided below.

{summary_contents}

From these summaries, compose a single lengthy, unified summary of the original text.
"""

output = model.invoke(
    prompt,
    model_kwargs={
        "max_tokens": 100000, # Set the maximum number of tokens to generate as output.
        "min_tokens": 5000, # Set the minimum number of tokens to generate as output.
        "temperature": 0.75,
        "system_prompt": "You are a helpful assistant.",
        "presence_penalty": 0,
        "frequency_penalty": 0
    }
    )

print(output)